In [0]:
'''
glo_crawler

README for 귤사모! - 상은

1) 주피터 노트북으로 돌리기

2) chrome webdriver 설치 필요: https://beomi.github.io/2017/02/27/HowToMakeWebCrawler-With-Selenium/ 설명 참고

3) 압축 해제 후 chromewebdriver.exe 를 C:// 경로에 넣어주면, 아래 코드를 수정 없이 사용 가능하고 permission 문제도 발생 X

4) https://www.glowpick.com/product/***** 에서 ***** = input !! ex) glo_crawler(*****)

5) 크롤링 데이터 저장시 한글/이모티콘 안 깨지게 인코딩 신경써주기: df.to_csv('경로', index=False, encoding = 'utf-8-sig')

'''

In [0]:
! pip install selenium

# glo_crawler, multi_glo_crawler, get_product_numbers

In [0]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

'''
################ 개별 제품 크롤링 ################
'''

def glo_crawler(product_number):
    
    # make dataframe
    columns = ['product', 'brand', 'age', 'skin', 'sex', 'rating', 'review']
    df = pd.DataFrame(columns=columns)
    
    # open window
    driver = webdriver.Chrome('C://chromedriver')
    driver.implicitly_wait(3)
    driver.get('https://www.glowpick.com/product/'+ str(product_number))
    
    table = driver.find_element_by_css_selector('#gp-default-main > section > div')
    # number of total reviews
    num_reviews = (table.find_element_by_css_selector('ul.contents__product-info > li.contents__product-info__li.contents__product-info__text > section.product-main-info > div.product-main-info__data_wrapper > div.product-main-info__ratings.ratings > span.ratings__review_count').text)[1:-1]
    if ',' in num_reviews:
        num_reviews = num_reviews.replace(',', '')
    num_reviews = int(num_reviews)

    # scrolling down
    num_of_pagedowns = 5  # round(num_reviews/20, 0)쓰려고 했는데... 리뷰 전부 못긁어가도록 한 100개까지만 로딩되게 해놓은듯...ㅠㅠ
    while num_of_pagedowns > 0:
        driver.find_element_by_tag_name('body').click()
        driver.find_element_by_tag_name('body').send_keys(Keys.END)
        time.sleep(1)
        num_of_pagedowns -= 1

    time.sleep(3)

    ### crawling data
    # product name, brand name, average rating
    product_name = table.find_element_by_css_selector('ul.contents__product-info > li.contents__product-info__li.contents__product-info__text > section.product-main-info > h1 > span').text
    brand_name = table.find_element_by_css_selector('ul.contents__product-info > li.contents__product-info__li.contents__product-info__text > section.product-main-info > p').text
    avg_rating = table.find_element_by_css_selector('ul.contents__product-info > li.contents__product-info__li.contents__product-info__text > section.product-main-info > div.product-main-info__data_wrapper > div.product-main-info__ratings.ratings > span.ratings__score').text
    avg_rating = float(avg_rating)

    print('제품명: ', product_name)
    print('브랜드명: ', brand_name)
    print('평균 점수: ', avg_rating)
    print('리뷰 개수: ', num_reviews)

    # information of reviewer(age, skin type, sex), rating, review text
    # 총 리뷰 수가 100개보다 많으면 100개 출력, 적으면 전부 출력
    if num_reviews > 100:
        k = 101
        
    else: k = num_reviews + 1
            
    for i in range(1,k):
        try:
            user_info = driver.find_element_by_xpath('//*[@id="gp-default-main"]/section/div/ul[2]/li[5]/section/ul/li[%s]/div/div/div/p/span[2]/span[1]'%i).text
            compiler = re.compile(r"(\d+)\w\s[·]\s(\w+)\s[·]")
            output = compiler.search(user_info)
        
            rating_info = table.find_element_by_css_selector('ul.contents__reviews > li.contents__reviews__li--right.contents__reviews__review-list > section > ul > li:nth-child(%s) > div > div > div > p > span.info > span.label > span'%i).get_attribute('class')
            compiler2 =re.compile(r"\w+[-]\w+\s\w+[-]\w+[-]\w+\s\w+[-](\w+)[-]\w+")
            output2 = compiler2.search(rating_info)
        
            review_dt = {
                'product': product_name,
                'brand': brand_name,
                'age': output.group(1),
                'skin': output.group(2),
                'sex': (table.find_element_by_css_selector('#gp-default-main > section > div > ul.contents__reviews > li.contents__reviews__li--right.contents__reviews__review-list > section > ul > li:nth-child(%s) > div > div > div > p > span.info > span.txt > span'%i).get_attribute('class'))[-1],
                'rating': output2.group(1),
                'review': table.find_element_by_css_selector('ul.contents__reviews > li.contents__reviews__li--right.contents__reviews__review-list > section > ul > li:nth-child(%s) > div > p'%i).text
            }
    
            df.loc[len(df)] = review_dt
            time.sleep(3)
            
        except: pass
        
    # rating에 대해 숫자로 mapping
    rating_mapping = {'best':5, 'good':4, 'soso':3, 'bad':2, 'worst':1}
    df['rating'] = df['rating'].map(rating_mapping)
    print('가져온 리뷰 개수: ', len(df))
    print('==============================')
    
    driver.close()
    
    return df

'''
################ 다수 제품 크롤링 ################
'''

def multi_glo_crawler(product_number_list):
    df = pd.DataFrame([])
    for i in product_number_list:
        try:
            data = glo_crawler(i)
            df = pd.concat([df, data], axis=0)
            time.sleep(10)
        except: 
            pass
    df.reset_index(drop=True, inplace=True)
    print('가져온 리뷰 개수 총합: ', len(df))
    return df


'''
################ 카테고리 랭킹 페이지에서 제품 번호 100개 가져오기 ################
'''
def get_product_numbers(url):
    driver = webdriver.Chrome('C://chromedriver')
    driver.implicitly_wait(3)
    driver.get(url)
    
    
    # scrolling down
    num_of_pagedowns = 5
    driver.find_element_by_tag_name('body').click()
    while num_of_pagedowns > 0:
        driver.find_element_by_tag_name('body').send_keys(Keys.END)
        time.sleep(3)
        num_of_pagedowns -= 1

    time.sleep(2)


    dt=[]
    for i in range(1,101):
        table = driver.find_element_by_css_selector('#gp-list > div > section.section-list > ul > li:nth-child(%s)'%i)
        url = table.find_element_by_css_selector('meta:nth-child(2)').get_attribute("content")
    
        compiler = re.compile(r"\w+[:][/][/]\w+[.]\w+[.]\w+[/]\w+[/](\d+)")
        output = int(compiler.search(url).group(1))
        dt.append(output)
    
    print('개수: ', len(dt))
    
    driver.close()

    return dt

# 함수 테스트

In [0]:
aa = glo_crawler(131757)
aa.head()

제품명:  유스 액티베이팅 셀 세럼
브랜드명:  헤라 (HERA)
평균 점수:  3.89
리뷰 개수:  88
가져온 리뷰 개수:  88


product      brand age skin sex  rating  \
0  유스 액티베이팅 셀 세럼  헤라 (HERA)  25   건성   f       4   
1  유스 액티베이팅 셀 세럼  헤라 (HERA)  25   건성   f       5   
2  유스 액티베이팅 셀 세럼  헤라 (HERA)  27   중성   f       4   
3  유스 액티베이팅 셀 세럼  헤라 (HERA)  28   건성   f       4   
4  유스 액티베이팅 셀 세럼  헤라 (HERA)  29   중성   f       5   

                                              review  
0  진짜 비싼게 좋긴하구나 싶었어요 ㅋㅋㅋㅋ 바르는데 피부에서 꽉 차오르는 느낌...?...  
1  헤라 샘플링 이벤트로 소량 사용해봤어요!!\n\n헤라 제품은 처음싸보는데 역시 비싼...  
2  역시 비싼건 비싼값을 하는군요..! 하루만 발라도 효과가 보인다는게 과장은 아니었어...  
3  한 세럼에 정착한 적 없고\n항상 더 좋은 애 없나 찾아다니는 유목민인데\n얘는 한...  
4  와우.. 사실 헤라 기초에는 큰 기대 없는 편인데 이번꺼는 진짜 좋은데요? 전 샘플...

In [0]:
num_list = [83967, 3524]
glowpick_data = multi_glo_crawler(num_list)
glowpick_data

제품명:  옴므 수퍼 아쿠아 크림
브랜드명:  헤라 (HERA)
평균 점수:  5.0
리뷰 개수:  2
가져온 리뷰 개수:  2
제품명:  세럼 브라이트 아이디어
브랜드명:  디티알티 (DTRT)
평균 점수:  5.0
리뷰 개수:  2
가져온 리뷰 개수:  2
가져온 리뷰 개수 총합:  4


product        brand age skin sex  rating  \
0  옴므 수퍼 아쿠아 크림    헤라 (HERA)  34  복합성   f       5   
1  옴므 수퍼 아쿠아 크림    헤라 (HERA)  29  복합성   f       5   
2  세럼 브라이트 아이디어  디티알티 (DTRT)  29  복합성   m       5   
3  세럼 브라이트 아이디어  디티알티 (DTRT)  26   건성   m       5   

                                              review  
0  향도좋고 발림성도좋고~ 가격도좋고 아버지께 선물드렸는데 엄청좋아하시더라고요~정말좋네...  
1  밖에 나왔는데 피부는 건조해 미칠것같고 화장안한상태에서 바를게 남자친구 수분크림밖에...  
2                     여름에 이것만 바르면 다해결될 정도고 냄새도좋고 좋아요  
3  여름에 야외에서 오래있었더니 피부가 너무 거매져서 미백세럼이라는 말에 샀어요. 액체...

# 크롤링 시작!

# suncream

In [0]:
url = 'https://www.glowpick.com/beauty/ranking?id=41&level=2&brand_category_id='
prod_list = get_product_numbers(url)
prod_list

개수:  100


[110260,
 117614,
 109087,
 106237,
 91399,
 121323,
 91668,
 22784,
 12236,
 89132,
 42801,
 77529,
 3416,
 54681,
 34779,
 120858,
 89131,
 13021,
 108714,
 123903,
 112507,
 4410,
 129984,
 105471,
 3702,
 34437,
 35505,
 122513,
 26725,
 35778,
 104620,
 22476,
 80754,
 68281,
 49033,
 12018,
 119259,
 34348,
 15808,
 73181,
 13139,
 22786,
 122403,
 77428,
 3220,
 68937,
 35586,
 104126,
 38610,
 27245,
 4412,
 10742,
 106066,
 122033,
 89461,
 79470,
 72607,
 104285,
 2933,
 11603,
 90869,
 106944,
 104468,
 11436,
 90516,
 89816,
 108116,
 19670,
 90839,
 109403,
 54521,
 120313,
 4359,
 34228,
 92175,
 4360,
 1001,
 105132,
 105590,
 93260,
 1921,
 105731,
 12421,
 17641,
 23709,
 105446,
 95265,
 118873,
 90571,
 6368,
 119216,
 105498,
 17640,
 57273,
 87271,
 25568,
 91069,
 38511,
 116803,
 89474]

In [0]:
l1 = prod_list[:10]
l2 = prod_list[10:20]
l3 = prod_list[20:30]
l4 = prod_list[30:40]
l5 = prod_list[40:50]
l6 = prod_list[50:60]
l7 = prod_list[60:70]
l8 = prod_list[70:80]
l9 = prod_list[80:90]
l10 = prod_list[90:100]

In [0]:
dt1 = multi_glo_crawler(l1)

제품명:  소프트 에어리 UV 에센스 [SPF50+/PA++++]
브랜드명:  클레어스 (dear, klairs)
평균 점수:  4.3
리뷰 개수:  478
가져온 리뷰 개수:  100
제품명:  UV 선프로 365 익스트림 선 플루이드 [SPF50+/PA++++]
브랜드명:  이자녹스 (ISAKNOX)
평균 점수:  4.19
리뷰 개수:  74
가져온 리뷰 개수:  74
제품명:  올 어라운드 세이프 블록 에센스 선 [SPF45/PA+++]
브랜드명:  미샤 (MISSHA)
평균 점수:  4.26
리뷰 개수:  87
가져온 리뷰 개수:  87
제품명:  워터풀 에센스 선크림 [SPF50+/PA++++]
브랜드명:  달바 (d'Alba)
평균 점수:  4.21
리뷰 개수:  90
가져온 리뷰 개수:  90
제품명:  파워블록 뽀송 선크림 [SPF50+/PA++++]
브랜드명:  어퓨 (Apieu)
평균 점수:  4.22
리뷰 개수:  93
가져온 리뷰 개수:  93
제품명:  릴리프 선 모이스처라이저 [SPF50+/PA++++]
브랜드명:  휘게 (HYGGEE)
평균 점수:  4.17
리뷰 개수:  109
가져온 리뷰 개수:  100
제품명:  유브이 퍼펙션 에브리데이선 [SPF50+/PA++++]
브랜드명:  더프트앤도프트 (DUFT&DOFT)
평균 점수:  4.16
리뷰 개수:  209
가져온 리뷰 개수:  100
제품명:  레이저 썬스크린100 [SPF50+/PA+++]
브랜드명:  셀퓨전씨 (Cell Fusion C)
평균 점수:  4.11
리뷰 개수:  936
가져온 리뷰 개수:  100
제품명:  유비데아 XL 멜트인 크림 [SPF50/PA++++]
브랜드명:  라로슈포제 (LA ROCHE-POSAY)
평균 점수:  4.11
리뷰 개수:  326
가져온 리뷰 개수:  100
제품명:  파워블록 에센스 선크림 [SPF50+/PA++++]
브랜드명:  어퓨 (Apieu)
평균 점수:  4.35
리뷰 개수:  54
가져온 리뷰 개수:  54
가져온 리뷰 

In [0]:
dt1.to_csv('C://Users//secho//Desktop//ss1.csv', index=False, encoding = 'utf-8-sig')

In [0]:
dt2 = multi_glo_crawler(l2)

제품명:  포에버 라이트 크리에이터 유브이 프로텍션 [SPF50+/PA+++]
브랜드명:  입생로랑 (YSL)
평균 점수:  4.1
리뷰 개수:  108
가져온 리뷰 개수:  100
제품명:  유브이 퍼펙션 하이드라선 [SPF50+/PA+++]
브랜드명:  더프트앤도프트 (DUFT&DOFT)
평균 점수:  4.1
리뷰 개수:  88
가져온 리뷰 개수:  88
제품명:  워터 스플래쉬 선크림 [SPF50+/PA+++]
브랜드명:  에스쁘아 (espoir)
평균 점수:  4.03
리뷰 개수:  1274
가져온 리뷰 개수:  100
제품명:  엔조이 마일드 선 에센스 [SPF50+/PA+++]
브랜드명:  싸이닉 (SCINIC)
평균 점수:  4.04
리뷰 개수:  1309
가져온 리뷰 개수:  100
제품명:  UV 엑스퍼트 XL쉴드 [SPF50/PA+++]
브랜드명:  랑콤 (LANCOME)
평균 점수:  4.06
리뷰 개수:  865
가져온 리뷰 개수:  100
제품명:  데일리 마일드 선스크린 [SPF50+/PA++++]
브랜드명:  이니스프리 (innisfree)
평균 점수:  4.1
리뷰 개수:  78
가져온 리뷰 개수:  78
제품명:  파워블록 데일리 선크림 [SPF50+/PA++++]
브랜드명:  어퓨 (Apieu)
평균 점수:  4.05
리뷰 개수:  253
가져온 리뷰 개수:  100
제품명:  메디 UV 울트라 선 [SPF50+/PA+++]
브랜드명:  Dr.G (닥터지)
평균 점수:  4.06
리뷰 개수:  421
가져온 리뷰 개수:  100
제품명:  이데베논 선블럭 [SPF50+/PA++++]
브랜드명:  울트라브이 (Ultra V)
평균 점수:  3.98
리뷰 개수:  115
가져온 리뷰 개수:  100
제품명:  오메가 레스큐 선크림 [SPF50+/PA+++]
브랜드명:  엑스와이지 포뮬러 (XYZ FORMULA)
평균 점수:  4.25
리뷰 개수:  52
가져온 리뷰 개수:  52
가져온 리뷰 개수 총합:  918


In [0]:
dt2.to_csv('C://Users//secho//Desktop//ss2.csv', index=False, encoding = 'utf-8-sig')

In [0]:
dt3 = multi_glo_crawler(l3)

제품명:  모이스트 UV 프로텍션 크림 윈터 베리어 [SPF50+/PA++++]
브랜드명:  이니스프리 (innisfree)
평균 점수:  4.07
리뷰 개수:  241
가져온 리뷰 개수:  100
제품명:  컨트롤 에이 센스티브 선 프로텍터 [SPF35/PA++]
브랜드명:  닥터자르트 (Dr.Jart)
평균 점수:  4.06
리뷰 개수:  873
가져온 리뷰 개수:  100
제품명:  세이프 블록 RX 올 파워풀 선 [SPF50+/PA++++]
브랜드명:  미샤 (MISSHA)
평균 점수:  4.13
리뷰 개수:  61
가져온 리뷰 개수:  61
제품명:  퓨어 미네랄 선크림 [SPF50+/PA++++]
브랜드명:  애즈이즈투비 (ASIS TOBE)
평균 점수:  4.12
리뷰 개수:  172
가져온 리뷰 개수:  100
제품명:  장조지롱 선블록 [SPF50+/PA+++]
브랜드명:  투쿨포스쿨 (too cool for school)
평균 점수:  3.99
리뷰 개수:  1129
가져온 리뷰 개수:  100
제품명:  엔조이 퍼펙트 데일리 선크림 [SPF50+/PA+++]
브랜드명:  싸이닉 (SCINIC)
평균 점수:  4.0
리뷰 개수:  807
가져온 리뷰 개수:  100
제품명:  UV 오일프리 선크림 [SPF50+/PA++++]
브랜드명:  에끌라두 (ECLADO)
평균 점수:  4.11
리뷰 개수:  171
가져온 리뷰 개수:  100
제품명:  블루베리 리밸런싱 워터리 선크림 [SPF45/PA+++]
브랜드명:  이니스프리 (innisfree)
평균 점수:  4.07
리뷰 개수:  169
가져온 리뷰 개수:  100
제품명:  세이프 앤 마일드 선 블록[SPF50+/PA+++]
브랜드명:  엔프라니 (ENPRANI)
평균 점수:  4.32
리뷰 개수:  56
가져온 리뷰 개수:  56
제품명:  유브이 어웨이 모이스트 점보 선블럭 [SPF50+/PA+++]
브랜드명:  잇츠스킨 (It'S SKIN)
평균 점수:  4.03
리뷰 개수:  92


In [0]:
dt3.to_csv('C://Users//secho//Desktop//ss3.csv', index=False, encoding = 'utf-8-sig')

In [0]:
dt4 = multi_glo_crawler(l4)

제품명:  시칼로에 릴리프 선밀크 [SPF50+/PA++++]
브랜드명:  랩노 (LabNo)
평균 점수:  4.16
리뷰 개수:  64
가져온 리뷰 개수:  64
제품명:  선크림 [SPF50+/PA+++]
브랜드명:  애터미 (atomy)
평균 점수:  3.95
리뷰 개수:  1999
가져온 리뷰 개수:  84
제품명:  내추럴 미네랄 썬 프로텍션
브랜드명:  라포티셀 (LAPOTHICELL)
평균 점수:  4.13
리뷰 개수:  62
가져온 리뷰 개수:  62
제품명:  선 프로텍트 앤 라이트 필 퍼펙트 선 세럼
브랜드명:  니베아 (NIVEA)
평균 점수:  4.07
리뷰 개수:  73
가져온 리뷰 개수:  73
제품명:  마일드 프로텍트 순한 선크림 [SPF50+/PA+++]
브랜드명:  구달 (goodal)
평균 점수:  4.02
리뷰 개수:  492
가져온 리뷰 개수:  100
제품명:  UV 플러스 데이 스크린HP [SPF40]
브랜드명:  클라랑스 (CLARINS)
평균 점수:  4.07
리뷰 개수:  187
가져온 리뷰 개수:  100
제품명:  퍼펙트 밸런스드 선스킨케어 [SPF50+/PA++++]
브랜드명:  유리프 (youlief)
평균 점수:  4.07
리뷰 개수:  58
가져온 리뷰 개수:  47
제품명:  베리 에센스 선블럭 [SPF50+/PA+++]
브랜드명:  본트리 (BORNTREE)
평균 점수:  4.03
리뷰 개수:  180
가져온 리뷰 개수:  100
제품명:  센서티브 마일드 선블록 [SPF35/PA++]
브랜드명:  CNP (씨앤피)
평균 점수:  3.99
리뷰 개수:  137
가져온 리뷰 개수:  100
제품명:  마일드 프로텍트 에센스 선 [SPF50+/PA++++]
브랜드명:  구달 (goodal)
평균 점수:  4.06
리뷰 개수:  62
가져온 리뷰 개수:  62
가져온 리뷰 개수 총합:  792


In [0]:
dt4.to_csv('C://Users//secho//Desktop//ss4.csv', index=False, encoding = 'utf-8-sig')

In [0]:
dt5 = multi_glo_crawler(l5)

제품명:  스킨 릴리프 UV 프로텍터 [SPF50+/PA+++]
브랜드명:  프리메라 (primera)
평균 점수:  3.95
리뷰 개수:  496
가져온 리뷰 개수:  100
제품명:  메디스파 선스크린 [SPF47/PA++]
브랜드명:  셀퓨전씨 (Cell Fusion C)
평균 점수:  4.15
리뷰 개수:  60
가져온 리뷰 개수:  60
제품명:  하이드로 멀티쉴드 [SPF50+/PA++++]
브랜드명:  로지컬리스킨 (Logically Skin)
평균 점수:  4.17
리뷰 개수:  60
가져온 리뷰 개수:  60
제품명:  데일리 선크림 [SPF50+/PA+++]
브랜드명:  아이뽀 (AIPPO)
평균 점수:  3.99
리뷰 개수:  78
가져온 리뷰 개수:  78
제품명:  UV 쉴드 선 프로텍터 [SPF50+/PA+++]
브랜드명:  아이오페 (IOPE)
평균 점수:  3.88
리뷰 개수:  1459
가져온 리뷰 개수:  100
제품명:  선 메이트 프로텍터 [SPF50+/PA+++]
브랜드명:  헤라 (HERA)
평균 점수:  3.93
리뷰 개수:  270
가져온 리뷰 개수:  100
제품명:  퍼펙트 에센스 선스크린 A+N [SPF50+/PA+++]
브랜드명:  아넷사 (ANESSA)
평균 점수:  3.97
리뷰 개수:  124
가져온 리뷰 개수:  100
제품명:  모이스쳐라이징 선 스크린 [SPF50+/PA+++]
브랜드명:  에슬레나 (eslena)
평균 점수:  4.02
리뷰 개수:  55
가져온 리뷰 개수:  55
제품명:  예브랑 센서티브 선크림 [SPF40/PA+++]
브랜드명:  화진 (HWAJIN)
평균 점수:  4.24
리뷰 개수:  49
가져온 리뷰 개수:  49
제품명:  선 프라이즈 마일드 에어리 피니쉬
브랜드명:  에뛰드 (ETUDE)
평균 점수:  3.92
리뷰 개수:  948
가져온 리뷰 개수:  100
가져온 리뷰 개수 총합:  802


In [0]:
dt5.to_csv('C://Users//secho//Desktop//ss5.csv', index=False, encoding = 'utf-8-sig')

In [0]:
dt6 = multi_glo_crawler(l6)

제품명:  에브리 선 데이 에센스 선
브랜드명:  닥터자르트 (Dr.Jart)
평균 점수:  3.95
리뷰 개수:  80
가져온 리뷰 개수:  30
제품명:  스마트솔루션365 실키 선블록 [SPF50/PA+++]
브랜드명:  잇츠스킨 (It'S SKIN)
평균 점수:  3.94
리뷰 개수:  99
가져온 리뷰 개수:  99
제품명:  플레쉬 업 선 [SPF50+/PA++++]
브랜드명:  미샤 (MISSHA)
평균 점수:  3.98
리뷰 개수:  57
가져온 리뷰 개수:  57
제품명:  그린티 시카 선로션 [SPF50+/PA++++]
브랜드명:  라운드어라운드 (ROUNDAROUND)
평균 점수:  3.89
리뷰 개수:  81
가져온 리뷰 개수:  81
제품명:  에코 어스 파워 노 세범 선 크림 [SPF50+/PA+++]
브랜드명:  더샘 (the SAEM)
평균 점수:  3.94
리뷰 개수:  117
가져온 리뷰 개수:  100
제품명:  프리미엄 선 프로텍션 크림 [SPF50+/PA+++]
브랜드명:  A24 (에이24)
평균 점수:  3.96
리뷰 개수:  94
가져온 리뷰 개수:  94
제품명:  알로에 썬프로텍션 크림
브랜드명:  오브레 (Obre)
평균 점수:  4.68
리뷰 개수:  31
가져온 리뷰 개수:  31
제품명:  에코 어스 파워 그린 선 크림 [SPF50+/PA++++]
브랜드명:  더샘 (the SAEM)
평균 점수:  4.17
리뷰 개수:  47
가져온 리뷰 개수:  47
제품명:  UV 엑스퍼트 GN-쉴드 자외선 차단제 [SPF50]
브랜드명:  랑콤 (LANCOME)
평균 점수:  3.93
리뷰 개수:  336
가져온 리뷰 개수:  100
제품명:  돈 하이드 선 에센스 [SPF37/PA++]
브랜드명:  데메테르 (DEMETER)
평균 점수:  4.23
리뷰 개수:  47
가져온 리뷰 개수:  47
가져온 리뷰 개수 총합:  686


In [0]:
dt6.to_csv('C://Users//secho//Desktop//ss6.csv', index=False, encoding = 'utf-8-sig')

In [0]:
dt7 = multi_glo_crawler(l7)

제품명:  2mg 선더마 B [SPF35/PA++]
브랜드명:  23 years old
평균 점수:  3.93
리뷰 개수:  56
가져온 리뷰 개수:  56
제품명:  포스트 알파 큐어 선스크린 100 [SPF50+/PA++++]
브랜드명:  셀퓨전씨 (Cell Fusion C)
평균 점수:  3.94
리뷰 개수:  101
가져온 리뷰 개수:  100
제품명:  익스트림 모이스처 선크림 [SPF50+/PA++++]
브랜드명:  리얼베리어 (Real Barrier)
평균 점수:  3.83
리뷰 개수:  88
가져온 리뷰 개수:  88
제품명:  선 어웨이 쿨링 워터리 선블럭
브랜드명:  숨37˚ (sum37˚)
평균 점수:  3.93
리뷰 개수:  68
가져온 리뷰 개수:  68
제품명:  트루케어 논나노 논코메도 무기자차 선크림 [SPF48/PA+++]
브랜드명:  이니스프리 (innisfree)
평균 점수:  3.84
리뷰 개수:  361
가져온 리뷰 개수:  100
제품명:  어딕트 미네랄 소프튼 프로텍트 선크림 [SPF50+/PA++++]
브랜드명:  쌍빠 (SAMPAR)
평균 점수:  3.85
리뷰 개수:  144
가져온 리뷰 개수:  100
제품명:  그린 UV 선 오프 [SPF50+/PA++++]
브랜드명:  비오엠 (BOM)
평균 점수:  3.9
리뷰 개수:  51
가져온 리뷰 개수:  51
제품명:  데일리 선 쉴드 크림 [SPF40++]
브랜드명:  이솔 (2SOL)
평균 점수:  3.88
리뷰 개수:  78
가져온 리뷰 개수:  78
제품명:  유브이 디펜스 미 블루 레이 선 플루이드 [SPF50+/PA++++]
브랜드명:  메이크프렘 (makeprem)
평균 점수:  3.86
리뷰 개수:  218
가져온 리뷰 개수:  100
제품명:  비트 더 선 [SPF47/PA++++]
브랜드명:  크레이브뷰티 (Krave Beauty)
평균 점수:  3.86
리뷰 개수:  81
가져온 리뷰 개수:  81
가져온 리뷰 개수 총합:  822


In [0]:
dt7.to_csv('C://Users//secho//Desktop//ss7.csv', index=False, encoding = 'utf-8-sig')

In [0]:
dt8 = multi_glo_crawler(l8)

제품명:  UV 페이스 프로텍터
브랜드명:  RMK (알엠케이)
평균 점수:  4.09
리뷰 개수:  58
가져온 리뷰 개수:  58
제품명:  톤업 노세범 선스크린 [SPF35/PA+++]
브랜드명:  이니스프리 (innisfree)
평균 점수:  3.86
리뷰 개수:  139
가져온 리뷰 개수:  100
제품명:  UV 프로텍터 스탠바이유 선스크린
브랜드명:  빌리프 (belif)
평균 점수:  3.84
리뷰 개수:  153
가져온 리뷰 개수:  100
제품명:  더마 디펜스 선 플루이드
브랜드명:  닥터자르트 (Dr.Jart)
평균 점수:  3.88
리뷰 개수:  397
가져온 리뷰 개수:  100
제품명:  퍼펙트 UV 선스크린 아쿠아부스터 [SPF50+/PA++++]
브랜드명:  아넷사 (ANESSA)
평균 점수:  3.87
리뷰 개수:  76
가져온 리뷰 개수:  76
제품명:  UV 프로텍터 멀티 선스크린
브랜드명:  빌리프 (belif)
평균 점수:  3.83
리뷰 개수:  140
가져온 리뷰 개수:  100
제품명:  퍼펙트 UV 선스크린 AA [SPF50+/PA+++]
브랜드명:  아넷사 (ANESSA)
평균 점수:  3.79
리뷰 개수:  523
가져온 리뷰 개수:  100
제품명:  울트라 라이트 데일리 유브이 디팬스 선스크린 [SPF50/PA++++]
브랜드명:  키엘 (Kiehl's)
평균 점수:  3.84
리뷰 개수:  95
가져온 리뷰 개수:  95
제품명:  365 안심 선크림 [SPF50+/PA++++]
브랜드명:  라운드랩 (ROUND LAB)
평균 점수:  3.86
리뷰 개수:  137
가져온 리뷰 개수:  100
제품명:  선크림 스테이 선 세이프 [SPF50+/PA++++]
브랜드명:  헉슬리 (Huxley)
평균 점수:  3.76
리뷰 개수:  68
가져온 리뷰 개수:  68
가져온 리뷰 개수 총합:  897


In [0]:
dt8.to_csv('C://Users//secho//Desktop//ss8.csv', index=False, encoding = 'utf-8-sig')

In [0]:
dt9 = multi_glo_crawler(l9)

제품명:  퓨어 블록 내추럴 데일리 선크림 [SPF45/PA+++]
브랜드명:  어퓨 (Apieu)
평균 점수:  3.75
리뷰 개수:  2653
가져온 리뷰 개수:  100
제품명:  헬로 써니 하이드레이팅 선 에센스 [SPF50+/PA++++]
브랜드명:  바닐라코 (BANILA CO)
평균 점수:  4.68
리뷰 개수:  25
가져온 리뷰 개수:  25
제품명:  365 선 크림 [SPF45/PA+++]
브랜드명:  이자녹스 (ISAKNOX)
평균 점수:  3.8
리뷰 개수:  158
가져온 리뷰 개수:  100
제품명:  트루 선블럭 [SPF50+/PA+++]
브랜드명:  라끄베르 (LACVERT)
평균 점수:  3.82
리뷰 개수:  154
가져온 리뷰 개수:  100
제품명:  민중기 무기자차 선크림 [SPF40/PA++]
브랜드명:  시드물 (SIDMOOL)
평균 점수:  3.76
리뷰 개수:  404
가져온 리뷰 개수:  100
제품명:  아임 쏘리 포 마이 스킨 데일리 카밍 선 크림 [SPF50+/PA++++]
브랜드명:  얼트루 (ultru)
평균 점수:  3.86
리뷰 개수:  51
가져온 리뷰 개수:  51
제품명:  선크림 베이지 [SPF50+/PA+++]
브랜드명:  애터미 (atomy)
평균 점수:  3.8
리뷰 개수:  108
가져온 리뷰 개수:  100
제품명:  UV 더마 무기자차 선크림 [SPF48/PA+++]
브랜드명:  닥터벨머 (Dr.Belmeur)
평균 점수:  3.71
리뷰 개수:  72
가져온 리뷰 개수:  72
제품명:  선 솔루션 프레쉬 선 에센스 IRF20 [SPF50+/PA++++]
브랜드명:  라비다 (LAVIDA)
평균 점수:  4.02
리뷰 개수:  51
가져온 리뷰 개수:  51
제품명:  이븐 베터 다크 스팟 디펜스 [SPF45/PA+++]
브랜드명:  크리니크 (CLINIQUE)
평균 점수:  3.78
리뷰 개수:  81
가져온 리뷰 개수:  24
가져온 리뷰 개수 총합:  723


In [0]:
dt9.to_csv('C://Users//secho//Desktop//ss9.csv', index=False, encoding = 'utf-8-sig')

In [0]:
dt10 = multi_glo_crawler(l10)

제품명:  UV 프로텍션 리바이탈 선 에센스 [SPF50+/PA++++]
브랜드명:  클라뷰 (KLAVUU)
평균 점수:  4.19
리뷰 개수:  31
가져온 리뷰 개수:  31
제품명:  선 메이트 엑설런스 [SPF50+/PA++++]
브랜드명:  헤라 (HERA)
평균 점수:  3.77
리뷰 개수:  60
가져온 리뷰 개수:  2
제품명:  선블락 포유 데일리 블락
브랜드명:  라끄베르 (LACVERT)
평균 점수:  3.92
리뷰 개수:  49
가져온 리뷰 개수:  49
제품명:  크레마카라콜 울트라 프로텍션 선크림 [SPF50+/PA+++]
브랜드명:  자민경
평균 점수:  4.42
리뷰 개수:  26
가져온 리뷰 개수:  26
제품명:  오트 마일드 모이스처 선크림 [SPF50+/PA+++]
브랜드명:  이니스프리 (innisfree)
평균 점수:  3.78
리뷰 개수:  393
가져온 리뷰 개수:  100
제품명:  무기자차 데일리 선밀크 [SPF20/PA++]
브랜드명:  시드물 (SIDMOOL)
평균 점수:  3.75
리뷰 개수:  65
가져온 리뷰 개수:  65
제품명:  알로에 워터리 쿨링 선크림 [SPF50+/PA++++]
브랜드명:  스킨푸드 (SKINFOOD)
평균 점수:  3.94
리뷰 개수:  48
가져온 리뷰 개수:  48
제품명:  오 마이 선 블럭 [SPF35/PA++]
브랜드명:  야다 (YADAH)
평균 점수:  3.88
리뷰 개수:  51
가져온 리뷰 개수:  51
제품명:  울트라 UV 쉴드 선 에센스 [SPF50+/PA++++]
브랜드명:  후르디아 (FRUDIA)
평균 점수:  4.17
리뷰 개수:  30
가져온 리뷰 개수:  30
제품명:  워터풀 선 에센스 [SPF50+/PA++++]
브랜드명:  보타닉힐 보 (BOTANIC HEAL boH)
평균 점수:  3.95
리뷰 개수:  39
가져온 리뷰 개수:  39
가져온 리뷰 개수 총합:  441


In [0]:
dt10.to_csv('C://Users//secho//Desktop//ss10.csv', index=False, encoding = 'utf-8-sig')

# sun stick

In [0]:
url = 'https://www.glowpick.com/beauty/ranking?id=43&level=2&brand_category_id='
prod_list = get_product_numbers(url)
prod_list

개수:  100


[118014,
 105129,
 106822,
 104924,
 119303,
 92371,
 122581,
 104675,
 91690,
 105319,
 104165,
 105551,
 106218,
 103923,
 120910,
 119105,
 104446,
 92111,
 106878,
 104139,
 105134,
 108304,
 105402,
 92162,
 107351,
 109414,
 104062,
 105131,
 120776,
 14561,
 92752,
 123522,
 106214,
 91229,
 90640,
 90758,
 105507,
 108196,
 91567,
 105894,
 8691,
 92585,
 93770,
 35787,
 108985,
 106794,
 105799,
 35097,
 104064,
 102006,
 105104,
 90744,
 109445,
 34174,
 34594,
 40062,
 91534,
 118115,
 109435,
 106465,
 123053,
 4356,
 108194,
 105403,
 106233,
 109491,
 109563,
 23094,
 74973,
 120779,
 89579,
 107656,
 105760,
 117540,
 34024,
 122551,
 124608,
 124274,
 123924,
 119453,
 117922,
 91037,
 121274,
 110772,
 109913,
 120372,
 121295,
 73697,
 109785,
 108597,
 119583,
 108198,
 122043,
 122121,
 103471,
 118632,
 119632,
 123829,
 122418,
 119768]

In [0]:
dt = multi_glo_crawler(prod_list)

제품명:  시카딘 센텔라 물스틱 선 [SPF50+/PA++++]
브랜드명:  미샤 (MISSHA)
평균 점수:  4.39
리뷰 개수:  287
가져온 리뷰 개수:  84
제품명:  흰감국 보송 선스틱 [SPF50+/PA++++]
브랜드명:  한율 (hanyul)
평균 점수:  4.33
리뷰 개수:  334
가져온 리뷰 개수:  100
제품명:  비타민 실키 선스틱 [SPF50+/PA++++]
브랜드명:  브링그린 (BRINGGREEN)
평균 점수:  4.27
리뷰 개수:  206
가져온 리뷰 개수:  100
제품명:  캘리포니아 알로에 보송 선스틱 [SPF50+/PA++++]
브랜드명:  네이처리퍼블릭 (NATURE REPUBLIC)
평균 점수:  4.31
리뷰 개수:  297
가져온 리뷰 개수:  100
제품명:  내추럴 퍼펙션 더블 쉴드 선스틱 [SPF50+/PA++++]
브랜드명:  AHC (에이에이치씨)
평균 점수:  4.23
리뷰 개수:  213
가져온 리뷰 개수:  100
제품명:  라이트 터치 선스틱 [SPF50+/PA++++]
브랜드명:  베리떼 (VERITE)
평균 점수:  4.18
리뷰 개수:  138
가져온 리뷰 개수:  100
제품명:  유브이 퍼펙션 쿨스틱 [SPF50+/PA++++]
브랜드명:  더프트앤도프트 (DUFT&DOFT)
평균 점수:  4.2
리뷰 개수:  60
가져온 리뷰 개수:  60
제품명:  에브리데이 선스틱 [SPF50+/PA++++]
브랜드명:  마몽드 (Mamonde)
평균 점수:  4.09
리뷰 개수:  94
가져온 리뷰 개수:  94
제품명:  유브이 퍼펙션 선스틱 [SPF50+/PA++++]
브랜드명:  더프트앤도프트 (DUFT&DOFT)
평균 점수:  3.98
리뷰 개수:  116
가져온 리뷰 개수:  100
제품명:  유브이 디펜스 미 블루 레이 선스틱 [SPF50+/PA++++]
브랜드명:  메이크프렘 (makeprem)
평균 점수:  4.01
리뷰 개수:  72
가져온 리뷰 개수:  72
제품명:  알

가져온 리뷰 개수:  25
제품명:  착한 선 스틱 [SPF50+/PA++++]
브랜드명:  블리블리 (VELY VELY)
평균 점수:  3.73
리뷰 개수:  11
가져온 리뷰 개수:  11
제품명:  그린 마일드 업 선스틱 [SPF50+/PA++++]
브랜드명:  Dr.G (닥터지)
평균 점수:  3.56
리뷰 개수:  18
가져온 리뷰 개수:  18
제품명:  오션 임프루브드 이지 선 스틱 알파 [SPF50+/PA++]
브랜드명:  한국화장품
평균 점수:  5.0
리뷰 개수:  3
가져온 리뷰 개수:  3
제품명:  365 선스틱 [SPF50+/PA+++]
브랜드명:  이자녹스 (ISAKNOX)
평균 점수:  3.86
리뷰 개수:  7
가져온 리뷰 개수:  7
제품명:  내츄럴 산뜻 파우더리 선스틱 [SPF50+/PA++++]
브랜드명:  온더바디 (ON THE BODY)
평균 점수:  4.25
리뷰 개수:  4
가져온 리뷰 개수:  4
제품명:  클리어 선스틱 [SPF50+/PA++++]
브랜드명:  홀리카홀리카 (HOLIKA HOLIKA)
평균 점수:  3.56
리뷰 개수:  27
가져온 리뷰 개수:  27
제품명:  이지워시 선스틱 [SPF47/PA+++]
브랜드명:  보타니컬테라피 (BOTANICAL THERAPY)
평균 점수:  4.0
리뷰 개수:  6
가져온 리뷰 개수:  6
제품명:  2IN1 선스틱 (Saturn) [SPF50+/PA++++]
브랜드명:  발드르 (BALDR)
평균 점수:  4.25
리뷰 개수:  4
가져온 리뷰 개수:  4
제품명:  데일리 유브이 프로텍션 스틱 칼라민 톤업 [SPF50+/PA++++]
브랜드명:  이니스프리 (innisfree)
평균 점수:  4.25
리뷰 개수:  4
가져온 리뷰 개수:  4
제품명:  아이디얼 솔레이스틱 [SPF50+/PA+++]
브랜드명:  비쉬 (VICHY)
평균 점수:  3.57
리뷰 개수:  14
가져온 리뷰 개수:  14
제품명:  올 데이 쉴드 선 스틱 [SPF50+/PA++

In [0]:
dt.to_csv('C://Users//secho//Desktop//glowpick_sunstick.csv', index=False, encoding = 'utf-8-sig')

# sun cushion

In [0]:
url = 'https://www.glowpick.com/beauty/ranking?id=43&level=2&brand_category_id='
prod_list = get_product_numbers(url)
dt = multi_glo_crawler(prod_list)

개수:  100
제품명:  시카딘 센텔라 물스틱 선 [SPF50+/PA++++]
브랜드명:  미샤 (MISSHA)
평균 점수:  4.39
리뷰 개수:  287
가져온 리뷰 개수:  100
제품명:  흰감국 보송 선스틱 [SPF50+/PA++++]
브랜드명:  한율 (hanyul)
평균 점수:  4.33
리뷰 개수:  334
가져온 리뷰 개수:  100
제품명:  비타민 실키 선스틱 [SPF50+/PA++++]
브랜드명:  브링그린 (BRINGGREEN)
평균 점수:  4.27
리뷰 개수:  206
가져온 리뷰 개수:  100
제품명:  캘리포니아 알로에 보송 선스틱 [SPF50+/PA++++]
브랜드명:  네이처리퍼블릭 (NATURE REPUBLIC)
평균 점수:  4.31
리뷰 개수:  297
가져온 리뷰 개수:  100
제품명:  내추럴 퍼펙션 더블 쉴드 선스틱 [SPF50+/PA++++]
브랜드명:  AHC (에이에이치씨)
평균 점수:  4.23
리뷰 개수:  213
가져온 리뷰 개수:  100
제품명:  라이트 터치 선스틱 [SPF50+/PA++++]
브랜드명:  베리떼 (VERITE)
평균 점수:  4.18
리뷰 개수:  138
가져온 리뷰 개수:  100
제품명:  유브이 퍼펙션 쿨스틱 [SPF50+/PA++++]
브랜드명:  더프트앤도프트 (DUFT&DOFT)
평균 점수:  4.2
리뷰 개수:  60
가져온 리뷰 개수:  60
제품명:  에브리데이 선스틱 [SPF50+/PA++++]
브랜드명:  마몽드 (Mamonde)
평균 점수:  4.09
리뷰 개수:  94
가져온 리뷰 개수:  94
제품명:  유브이 퍼펙션 선스틱 [SPF50+/PA++++]
브랜드명:  더프트앤도프트 (DUFT&DOFT)
평균 점수:  3.98
리뷰 개수:  116
가져온 리뷰 개수:  100
제품명:  유브이 디펜스 미 블루 레이 선스틱 [SPF50+/PA++++]
브랜드명:  메이크프렘 (makeprem)
평균 점수:  4.01
리뷰 개수:  72
가져온 리뷰 개수:  

가져온 리뷰 개수:  11
제품명:  그린 마일드 업 선스틱 [SPF50+/PA++++]
브랜드명:  Dr.G (닥터지)
평균 점수:  3.56
리뷰 개수:  18
가져온 리뷰 개수:  18
제품명:  오션 임프루브드 이지 선 스틱 알파 [SPF50+/PA++]
브랜드명:  한국화장품
평균 점수:  5.0
리뷰 개수:  3
가져온 리뷰 개수:  3
제품명:  365 선스틱 [SPF50+/PA+++]
브랜드명:  이자녹스 (ISAKNOX)
평균 점수:  3.86
리뷰 개수:  7
가져온 리뷰 개수:  7
제품명:  내츄럴 산뜻 파우더리 선스틱 [SPF50+/PA++++]
브랜드명:  온더바디 (ON THE BODY)
평균 점수:  4.25
리뷰 개수:  4
가져온 리뷰 개수:  4
제품명:  클리어 선스틱 [SPF50+/PA++++]
브랜드명:  홀리카홀리카 (HOLIKA HOLIKA)
평균 점수:  3.56
리뷰 개수:  27
가져온 리뷰 개수:  27
제품명:  이지워시 선스틱 [SPF47/PA+++]
브랜드명:  보타니컬테라피 (BOTANICAL THERAPY)
평균 점수:  4.0
리뷰 개수:  6
가져온 리뷰 개수:  6
제품명:  2IN1 선스틱 (Saturn) [SPF50+/PA++++]
브랜드명:  발드르 (BALDR)
평균 점수:  4.25
리뷰 개수:  4
가져온 리뷰 개수:  4
제품명:  데일리 유브이 프로텍션 스틱 칼라민 톤업 [SPF50+/PA++++]
브랜드명:  이니스프리 (innisfree)
평균 점수:  4.25
리뷰 개수:  4
가져온 리뷰 개수:  4
제품명:  아이디얼 솔레이스틱 [SPF50+/PA+++]
브랜드명:  비쉬 (VICHY)
평균 점수:  3.57
리뷰 개수:  14
가져온 리뷰 개수:  14
제품명:  올 데이 쉴드 선 스틱 [SPF50+/PA+++]
브랜드명:  이솔 (2SOL)
평균 점수:  4.25
리뷰 개수:  4
가져온 리뷰 개수:  4
제품명:  더마 카밍 선스틱[SPF50+/PA++++]
브랜드명:

In [0]:
dt.to_csv('C://Users//secho//Desktop//glowpick_suncushion.csv', index=False, encoding = 'utf-8-sig')

# sun gel

In [0]:
url = 'https://www.glowpick.com/beauty/ranking?id=44&level=2&brand_category_id='
prod_list = get_product_numbers(url)
dt = multi_glo_crawler(prod_list)

개수:  100
제품명:  산소수 라이트 선 젤 [SPF50+/PA++++]
브랜드명:  식물나라 (SHINGMULNARA)
평균 점수:  4.25
리뷰 개수:  1434
가져온 리뷰 개수:  100
제품명:  리얼 하이알루로닉 캡슐 선 젤 [SPF50+/PA++++]
브랜드명:  웰라쥬 (WELLAGE)
평균 점수:  4.24
리뷰 개수:  93
가져온 리뷰 개수:  91
제품명:  올 어라운드 세이프 블록 아쿠아 선 젤 [SPF50+/PA++++]
브랜드명:  미샤 (MISSHA)
평균 점수:  4.24
리뷰 개수:  190
가져온 리뷰 개수:  100
제품명:  선 프로텍트 워터젤 [SPF50/PA+++]
브랜드명:  니베아 (NIVEA)
평균 점수:  4.23
리뷰 개수:  261
가져온 리뷰 개수:  100
제품명:  UV 아쿠아 리치 워터리 젤 [SPF50+/PA++++]
브랜드명:  비오레 (Biore)
평균 점수:  4.15
리뷰 개수:  143
가져온 리뷰 개수:  100
제품명:  비타민 워터리 선 젤 [SPF50+/PA++++]
브랜드명:  브링그린 (BRINGGREEN)
평균 점수:  4.07
리뷰 개수:  340
가져온 리뷰 개수:  100
제품명:  UV 퍼펙트 에어쿨 선 젤[SPF50+/PA++++]
브랜드명:  SNP (에스앤피)
평균 점수:  4.06
리뷰 개수:  180
가져온 리뷰 개수:  100
제품명:  UV 엑스퍼트 유스 쉴드 아쿠아 젤
브랜드명:  랑콤 (LANCOME)
평균 점수:  4.14
리뷰 개수:  80
가져온 리뷰 개수:  80
제품명:  유브이 더블 컷 산뜻 선 젤 [SPF50+/PA++++]
브랜드명:  에뛰드 (ETUDE)
평균 점수:  4.03
리뷰 개수:  473
가져온 리뷰 개수:  100
제품명:  알로에 워터프루프 선 젤 [SPF50+/PA++++]
브랜드명:  홀리카홀리카 (HOLIKA HOLIKA)
평균 점수:  4.0
리뷰 개수:  268
가져온 리뷰 개수:  100
제품명:  퍼펙셔니스트

제품명:  선 프로텍트 워터 젤
브랜드명:  니베아 (NIVEA)
평균 점수:  4.0
리뷰 개수:  2
가져온 리뷰 개수:  2
제품명:  에브리데이 선 젤 [SPF50+/PA++++]
브랜드명:  어퓨 (Apieu)
평균 점수:  4.0
리뷰 개수:  2
가져온 리뷰 개수:  2
제품명:  누드 앤 쉬어 선 젤 퓨어 [SPF50+/PA+++]
브랜드명:  위드미 (WITHME)
평균 점수:  5.0
리뷰 개수:  1
가져온 리뷰 개수:  1
제품명:  데일리 라이트핏 선젤 [SPF50+/PA+++]
브랜드명:  다슈 (DASHU)
평균 점수:  5.0
리뷰 개수:  1
가져온 리뷰 개수:  1
제품명:  엔젤 선블럭 [SPF50+/PA+++]
브랜드명:  엔젤 (ANGELS)
평균 점수:  5.0
리뷰 개수:  1
가져온 리뷰 개수:  1
제품명:  에코 데일리 디펜스 아쿠아 선젤 [SPF42/PA++]
브랜드명:  비욘드 (BEYOND)
평균 점수:  5.0
리뷰 개수:  1
가져온 리뷰 개수:  1
제품명:  알로에 선젤 (바이오 쿨 민트 선 젤) [SPF50+/PA+++]
브랜드명:  스와니코코 (swanicoco)
평균 점수:  3.6
리뷰 개수:  5
가져온 리뷰 개수:  5
제품명:  워터쿨 디펜스 선 젤 [SPF50+/PA++++]
브랜드명:  보타닉힐 보 (BOTANIC HEAL boH)
평균 점수:  3.63
리뷰 개수:  8
가져온 리뷰 개수:  8
제품명:  솔라케어 90 겔 플루이드 크림 [SPF50+/PA+++]
브랜드명:  써모슈티컬 (tHermo CEUTICAL)
평균 점수:  4.0
리뷰 개수:  2
가져온 리뷰 개수:  2
제품명:  파워브록 워터프루프 프레쉬 선 젤 [SPF50+/PA++++]
브랜드명:  어퓨 (Apieu)
평균 점수:  3.5
리뷰 개수:  2
가져온 리뷰 개수:  2
제품명:  어반 폴루아웃 선 젤 [SPF50+/PA+++]
브랜드명:  뮤즈베라 (MUSE VERA)
평균 점수:  4.0
리뷰 개수:  

In [0]:
dt.to_csv('C://Users//secho//Desktop//glowpick_sungel.csv', index=False, encoding = 'utf-8-sig')

# sun base

In [0]:
url = 'https://www.glowpick.com/beauty/ranking?id=45&level=2&brand_category_id='
prod_list = get_product_numbers(url)
dt = multi_glo_crawler(prod_list)

개수:  100
제품명:  프로 엑스트라 프로텍티브 선베이스 [SPF50+/PA+++]
브랜드명:  에스쁘아 (espoir)
평균 점수:  4.03
리뷰 개수:  172
가져온 리뷰 개수:  100
제품명:  에브리 선데이
브랜드명:  닥터자르트 (Dr.Jart)
평균 점수:  4.03
리뷰 개수:  269
가져온 리뷰 개수:  100
제품명:  유브이 퍼펙션 멀티베이스 [SPF50+/PA+++]
브랜드명:  더프트앤도프트 (DUFT&DOFT)
평균 점수:  3.92
리뷰 개수:  140
가져온 리뷰 개수:  100
제품명:  피토 아쿠아 워터 선 베이스 [SPF50+/PA+++]
브랜드명:  비욘드 (BEYOND)
평균 점수:  3.96
리뷰 개수:  565
가져온 리뷰 개수:  100
제품명:  유비데아 XL 멜트-인 틴티드 크림 [SPF50+/PA++++]
브랜드명:  라로슈포제 (LA ROCHE-POSAY)
평균 점수:  3.97
리뷰 개수:  144
가져온 리뷰 개수:  100
제품명:  포어 애플 선블록 [SPF50+/PA+++]
브랜드명:  입큰 (IPKN)
평균 점수:  3.88
리뷰 개수:  72
가져온 리뷰 개수:  72
제품명:  선 파우더 로션 [SPF25/PA+++]
브랜드명:  사바비안 (Ca va bien)
평균 점수:  3.96
리뷰 개수:  76
가져온 리뷰 개수:  76
제품명:  멀티 톤 업 에센셜 선크림 [SPF50+/PA++++]
브랜드명:  그라운드플랜 (GROUND PLAN)
평균 점수:  3.85
리뷰 개수:  54
가져온 리뷰 개수:  54
제품명:  UV 엑스퍼트 GN-쉴드 메이크업 베이스 [SPF50/PA+++]
브랜드명:  랑콤 (LANCOME)
평균 점수:  3.87
리뷰 개수:  113
가져온 리뷰 개수:  100
제품명:  메이크업 선 크림 [SPF50+/PA+++]
브랜드명:  홀리카홀리카 (HOLIKA HOLIKA)
평균 점수:  3.85
리뷰 개수:  401
가져온 리뷰 개수:  100
제품명:  아

가져온 리뷰 개수:  1
제품명:  스킨 세팅 워터 선 베이스 [SPF 50+/PA ++++]
브랜드명:  정샘물 (JUNGSAEMMOOL)
평균 점수:  5.0
리뷰 개수:  1
가져온 리뷰 개수:  1
제품명:  아쿠아 선 EX [SPF50+/PA+++]
브랜드명:  캐시캣 (Cathy Cat)
평균 점수:  5.0
리뷰 개수:  1
가져온 리뷰 개수:  1
제품명:  윤광 플라워 선밀크 로즈
브랜드명:  JM솔루션 (JMsolution)
평균 점수:  5.0
리뷰 개수:  1
가져온 리뷰 개수:  1
제품명:  이븐 베터 시티 블록 안티-폴루션 [SPF40/PA+++]
브랜드명:  크리니크 (CLINIQUE)
평균 점수:  3.5
리뷰 개수:  18
가져온 리뷰 개수:  18
제품명:  이지선 벨벳프라이머 [SPF40/PA+++]
브랜드명:  이지듀 (Easydew)
평균 점수:  3.67
리뷰 개수:  3
가져온 리뷰 개수:  3
제품명:  프라임 프로텍트 선 베이스 [SPF50+/PA++++]
브랜드명:  포니이펙트 (PONY EFFECT)
평균 점수:  3.67
리뷰 개수:  3
가져온 리뷰 개수:  3
제품명:  UV 선 블록 크림 [SPF40/PA++]
브랜드명:  크리스찬지오 (Christian Gio)
평균 점수:  4.0
리뷰 개수:  2
가져온 리뷰 개수:  2
제품명:  더마 베이스톤업 아쿠아 선블럭
브랜드명:  라보케어 (LABOCARE)
평균 점수:  4.0
리뷰 개수:  2
가져온 리뷰 개수:  2
제품명:  아임톤업 멀티베이스 [SPF50+/PA++++]
브랜드명:  선티크 (SUNTIQUE)
평균 점수:  4.0
리뷰 개수:  2
가져온 리뷰 개수:  2
제품명:  UV틴트 밀크선크림 [SPF50+/PA++]
브랜드명:  아크네스 (Acnes)
평균 점수:  3.6
리뷰 개수:  5
가져온 리뷰 개수:  5
제품명:  캘리포니안 네이처 프리미엄 멀티 프로텍션 크림
브랜드명:  A24 (에이24)
평균 점수:  5.0
리뷰 개수:

In [0]:
dt.to_csv('C://Users//secho//Desktop//glowpick_sunbase.csv', index=False, encoding = 'utf-8-sig')

# sun spray

In [0]:
url = 'https://www.glowpick.com/beauty/ranking?id=46&level=2&brand_category_id='
prod_list = get_product_numbers(url)
dt = multi_glo_crawler(prod_list)

개수:  100
제품명:  산소수 아이스 선 스프레이 [SPF50+/PA+++]
브랜드명:  식물나라 (SHINGMULNARA)
평균 점수:  3.87
리뷰 개수:  274
가져온 리뷰 개수:  100
제품명:  크리스탈 선 스프레이 AD [SPF50+/PA+++]
브랜드명:  레시피 (RECIPE)
평균 점수:  4.0
리뷰 개수:  14
가져온 리뷰 개수:  14
제품명:  내추럴 리프레쉬 선스프레이
브랜드명:  온더바디 (ON THE BODY)
평균 점수:  3.59
리뷰 개수:  27
가져온 리뷰 개수:  27
제품명:  퍼펙트 에센스 스프레이
브랜드명:  아넷사 (ANESSA)
평균 점수:  4.11
리뷰 개수:  9
가져온 리뷰 개수:  9
제품명:  산소수 워터프루프 선 스프레이 [SPF50+/PA+++]
브랜드명:  식물나라 (SHINGMULNARA)
평균 점수:  3.39
리뷰 개수:  138
가져온 리뷰 개수:  100
제품명:  칙칙 썬스프레이
브랜드명:  데메테르 (DEMETER)
평균 점수:  4.5
리뷰 개수:  4
가져온 리뷰 개수:  4
제품명:  투명 쿨링 선스프레이
브랜드명:  온더바디 (ON THE BODY)
평균 점수:  4.2
리뷰 개수:  5
가져온 리뷰 개수:  5
제품명:  포토덤 맥스 스프레이 [SPF50+]
브랜드명:  바이오더마 (BIODERMA)
평균 점수:  4.0
리뷰 개수:  7
가져온 리뷰 개수:  7
제품명:  데이블럭 선 쉴드 미스트 [SPF50+/PA+++]
브랜드명:  암웨이 (Amway)
평균 점수:  4.0
리뷰 개수:  4
가져온 리뷰 개수:  4
제품명:  선 프로텍션 벨벳 미스트 선 스프레이 EX [SPF50+/PA+++]
브랜드명:  오휘 (O HUI)
평균 점수:  3.57
리뷰 개수:  7
가져온 리뷰 개수:  7
제품명:  알로에 아이스 쿨링 선 스프레이 [SPF50+/PA++++]
브랜드명:  홀리카홀리카 (HOLIKA HOLIKA)
평균 점수:  4.5
리뷰 개수:  2
가져온

가져온 리뷰 개수:  18
제품명:  수분가득 선 스프레이 [SPF29/PA++]
브랜드명:  시드물 (SIDMOOL)
평균 점수:  2.38
리뷰 개수:  8
가져온 리뷰 개수:  8
제품명:  울트라 쉬어 웻스킨 선스크린 바디 미스트 [SPF50+/PA+++]
브랜드명:  뉴트로지나 (Neutrogena)
평균 점수:  2.79
리뷰 개수:  19
가져온 리뷰 개수:  19
제품명:  제트쏠레 썬 스프레이
브랜드명:  오제끄 (OSEQUE)
평균 점수:  2.22
리뷰 개수:  9
가져온 리뷰 개수:  9
제품명:  마이써니 올 오버 선 스프레이 [SPF50+/PA+++]
브랜드명:  토니모리 (TONYMOLY)
평균 점수:  3.06
리뷰 개수:  50
가져온 리뷰 개수:  50
제품명:  프리미엄 토마토 쿨 앤 드라이 선스프레이 [SPF50+/PA+++]
브랜드명:  스킨푸드 (SKINFOOD)
평균 점수:  2.42
리뷰 개수:  12
가져온 리뷰 개수:  12
제품명:  퓨어 앤 센서티브 선 스프레이 [SPF50+/PA+++]
브랜드명:  니베아 (NIVEA)
평균 점수:  2.0
리뷰 개수:  9
가져온 리뷰 개수:  9
제품명:  마이써니 클리어 선 스프레이 [SPF50+/PA+++]
브랜드명:  토니모리 (TONYMOLY)
평균 점수:  2.71
리뷰 개수:  24
가져온 리뷰 개수:  24
제품명:  아쿠아 프로텍트 선 스프레이 [SPF30/PA++]
브랜드명:  니베아 (NIVEA)
평균 점수:  2.0
리뷰 개수:  10
가져온 리뷰 개수:  10
제품명:  마이써니 올 선스프레이 [SPF50/PA+++]
브랜드명:  토니모리 (TONYMOLY)
평균 점수:  2.5
리뷰 개수:  16
가져온 리뷰 개수:  16
제품명:  썬 프로텍트 앤 리프레쉬 쿨링 미스트
브랜드명:  니베아 (NIVEA)
평균 점수:  2.36
리뷰 개수:  14
가져온 리뷰 개수:  14
제품명:  제트쏠레 UV프로텍트 선 스프레이 [SPF50+/PA+++]
브랜드

In [0]:
dt.to_csv('C://Users//secho//Desktop//glowpick_sunspray.csv', index=False, encoding = 'utf-8-sig')

# 파데

In [0]:
url = 'https://www.glowpick.com/beauty/ranking?id=7&level=2&brand_category_id='
prod_list = get_product_numbers(url)
dt1 = multi_glo_crawler(prod_list)

개수:  100
제품명:  윙크 파운데이션 팩트 [SPF50+/PA+++]
브랜드명:  자빈드서울 (JAVIN DE SEOUL)
평균 점수:  4.35
리뷰 개수:  179
가져온 리뷰 개수:  100
제품명:  인텐시브 스킨 세럼 파운데이션 [SPF40/PA+++]
브랜드명:  바비브라운 (BOBBI BROWN)
평균 점수:  4.34
리뷰 개수:  155
가져온 리뷰 개수:  100
제품명:  매트 벨벳 스킨 블러링 파우더 파운데이션
브랜드명:  메이크업포에버 (MAKE UP FOR EVER)
평균 점수:  4.26
리뷰 개수:  130
가져온 리뷰 개수:  100
제품명:  아우라 워터 글로우 쿠션 [SPF50+/PA+++]
브랜드명:  룩스앤메이 코스메틱 (LOOKS&MEII COSMETIC)
평균 점수:  4.17
리뷰 개수:  109
가져온 리뷰 개수:  46
제품명:  새틴 핏 레이어 쿠션 [SPF50/PA++++]
브랜드명:  조성아22
평균 점수:  4.22
리뷰 개수:  153
가져온 리뷰 개수:  25
제품명:  래디언스 퍼펙트핏 파운데이션 [SPF30/PA++]
브랜드명:  미샤 (MISSHA)
평균 점수:  4.14
리뷰 개수:  166
가져온 리뷰 개수:  37
제품명:  퓨처리스트 아쿠아 브릴리언스 파운데이션 [SPF20/PA+++]
브랜드명:  에스티 로더 (ESTEE LAUDER)
평균 점수:  4.12
리뷰 개수:  2802
가져온 리뷰 개수:  83
제품명:  마이크로 글래시핏 쿠션 [SPF50+/PA++++]
브랜드명:  문샷 (moonshot)
평균 점수:  4.14
리뷰 개수:  224
가져온 리뷰 개수:  100
제품명:  포에버 스킨 글로우 24H 웨어 스킨 케어링 파운데이션 [SPF35/PA++]
브랜드명:  디올 (DIOR)
평균 점수:  4.18
리뷰 개수:  140
가져온 리뷰 개수:  1
제품명:  뗑 이돌 롱라스팅 파운데이션 [SPF38/PA++]
브랜드명:  랑콤 (LANCOME)
평균 점수:  4.09


제품명:  마이크로 세팅핏 쿠션 [SPF50+/PA+++]
브랜드명:  문샷 (moonshot)
평균 점수:  3.89
리뷰 개수:  643
가져온 리뷰 개수:  100
제품명:  파우더리 스킨 메이커
브랜드명:  케이트 (KATE)
평균 점수:  3.91
리뷰 개수:  361
가져온 리뷰 개수:  100
제품명:  스튜디오 드 땅뜨 에끌라 새틴 쿠션
브랜드명:  투쿨포스쿨 (too cool for school)
평균 점수:  3.84
리뷰 개수:  427
가져온 리뷰 개수:  18
제품명:  더마 인텐시브 판테놀 앰플 쿠션
브랜드명:  보타닉힐 보 (BOTANIC HEAL boH)
평균 점수:  3.87
리뷰 개수:  89
가져온 리뷰 개수:  89
제품명:  라이트 벨벳 쿠션 [SPF50+/PA+++]
브랜드명:  메이크업포에버 (MAKE UP FOR EVER)
평균 점수:  3.91
리뷰 개수:  231
가져온 리뷰 개수:  75
제품명:  프로 롱웨어 너리싱 워터프루프 파운데이션
브랜드명:  맥 (MAC)
평균 점수:  3.88
리뷰 개수:  784
가져온 리뷰 개수:  1
제품명:  프로 피니쉬
브랜드명:  메이크업포에버 (MAKE UP FOR EVER)
평균 점수:  3.83
리뷰 개수:  708
가져온 리뷰 개수:  100
제품명:  아트클래스 스튜디오 드 땅뜨 글로우 커버 쿠션 [SPF50+/PA+++]
브랜드명:  투쿨포스쿨 (too cool for school)
평균 점수:  3.86
리뷰 개수:  835
가져온 리뷰 개수:  100
제품명:  크리스탈 컨실러 쿠션
브랜드명:  낫츠 (NOTS)
평균 점수:  3.86
리뷰 개수:  76
가져온 리뷰 개수:  76
제품명:  아트클래스 스튜디오 드 땅뜨 픽싱 커버 쿠션 [SPF45/PA++]
브랜드명:  투쿨포스쿨 (too cool for school)
평균 점수:  3.9
리뷰 개수:  410
가져온 리뷰 개수:  100
제품명:  UV 리퀴드 파운데이션
브랜드명:  RMK (알엠케이)
평균

In [0]:
dt1.to_csv('C://Users//secho//Desktop//glowpick_foundation.csv', index=False, encoding = 'utf-8-sig')

# 파우더

In [0]:
url = 'https://www.glowpick.com/beauty/ranking?id=8&level=2&brand_category_id='
prod_list = get_product_numbers(url)
dt2 = multi_glo_crawler(prod_list)

개수:  100
제품명:  울트라 HD 프레스드 파우더
브랜드명:  메이크업포에버 (MAKE UP FOR EVER)
평균 점수:  4.49
리뷰 개수:  1319
가져온 리뷰 개수:  40
제품명:  울트라 HD 파우더
브랜드명:  메이크업포에버 (MAKE UP FOR EVER)
평균 점수:  4.5
리뷰 개수:  595
가져온 리뷰 개수:  100
제품명:  코튼 베일 파우더
브랜드명:  마몽드 (Mamonde)
평균 점수:  4.41
리뷰 개수:  1608
가져온 리뷰 개수:  100
제품명:  라이트 리플렉팅 루스 세팅 파우더
브랜드명:  나스 (NARS)
평균 점수:  4.42
리뷰 개수:  99
가져온 리뷰 개수:  20
제품명:  프라임 프라이머 피니쉬 파우더
브랜드명:  바닐라코 (BANILA CO)
평균 점수:  4.48
리뷰 개수:  102
가져온 리뷰 개수:  100
제품명:  루스 세팅 파우더
브랜드명:  로라메르시에 (LAURA MERCIER)
평균 점수:  4.33
리뷰 개수:  349
가져온 리뷰 개수:  100
제품명:  포어 블러 파우더
브랜드명:  이니스프리 (innisfree)
평균 점수:  4.38
리뷰 개수:  221
가져온 리뷰 개수:  100
제품명:  피치뽀송 멀티 피니시 파우더
브랜드명:  스킨푸드 (SKINFOOD)
평균 점수:  4.29
리뷰 개수:  2453
가져온 리뷰 개수:  100
제품명:  노세범 미네랄 파우더
브랜드명:  이니스프리 (innisfree)
평균 점수:  4.32
리뷰 개수:  17486
가져온 리뷰 개수:  100
제품명:  미네랄 피니싱 파우더
브랜드명:  로라메르시에 (LAURA MERCIER)
평균 점수:  4.37
리뷰 개수:  161
가져온 리뷰 개수:  100
제품명:  시크릿 뷰티 파우더
브랜드명:  캔메이크 (CANMAKE)
평균 점수:  4.31
리뷰 개수:  145
가져온 리뷰 개수:  100
제품명:  미네랄 100 HD 파우더
브랜드명:  어퓨 (Apieu)
평균 점수

가져온 리뷰 개수:  34
제품명:  데미지 클리닉 린스
브랜드명:  케라시스 (Kerasys)
평균 점수:  3.7
리뷰 개수:  64
가져온 리뷰 개수:  64
제품명:  라이트 핏 파우더
브랜드명:  라네즈 (LANEIGE)
평균 점수:  4.46
리뷰 개수:  13
가져온 리뷰 개수:  13
제품명:  쎌루라 트리트먼트 루스 파우더
브랜드명:  라프레리 (la prairie)
평균 점수:  3.78
리뷰 개수:  36
가져온 리뷰 개수:  20
제품명:  오일 컨트롤 필름 파우더 [SPF15]
브랜드명:  어퓨 (Apieu)
평균 점수:  3.49
리뷰 개수:  191
가져온 리뷰 개수:  100
제품명:  엔젤 페이스 핑크 파우더
브랜드명:  폰즈 (POND'S)
평균 점수:  4.0
리뷰 개수:  17
가져온 리뷰 개수:  17
제품명:  에코 소울 퍼펙트 커버 팩트
브랜드명:  더샘 (the SAEM)
평균 점수:  4.12
리뷰 개수:  17
가져온 리뷰 개수:  16
제품명:  노세범 블러 팩트
브랜드명:  이니스프리 (innisfree)
평균 점수:  3.52
리뷰 개수:  326
가져온 리뷰 개수:  100
제품명:  스킨 파운데이션 미네랄 메이크업 [SPF15]
브랜드명:  바비브라운 (BOBBI BROWN)
평균 점수:  3.95
리뷰 개수:  22
가져온 리뷰 개수:  22
제품명:  하이드라 미스트 세트 & 리프레시 파우더
브랜드명:  베카 (BECCA)
평균 점수:  4.88
리뷰 개수:  8
가져온 리뷰 개수:  8
제품명:  스킨 라이트 프레스토 파우더
브랜드명:  레브론 (REVLON)
평균 점수:  4.5
리뷰 개수:  10
가져온 리뷰 개수:  10
제품명:  AQMW 페이스 파우더
브랜드명:  코스메데코르테 (COSME DECORTE)
평균 점수:  4.75
리뷰 개수:  8
가져온 리뷰 개수:  8
제품명:  피치뽀송 포어 선팩트 [SPF42/PA+++]
브랜드명:  스킨푸드 (SKINFOOD)
평균 점수:  3.54


In [0]:
dt2.to_csv('C://Users//secho//Desktop//glowpick_finishpowder.csv', index=False, encoding = 'utf-8-sig')

# 제품 리스트에서 product number 추출

In [0]:
url = 'https://www.glowpick.com/beauty/ranking?id=14&level=2&brand_category_id='
get_product_numbers(url) #1

개수:  100


[49097,
 111771,
 80915,
 97495,
 85963,
 71933,
 9783,
 21586,
 116516,
 117343,
 117648,
 128938,
 95456,
 90,
 96648,
 80276,
 110168,
 89691,
 93182,
 91,
 13971,
 110211,
 85036,
 117886,
 98856,
 124954,
 4924,
 62977,
 130647,
 78818,
 129123,
 124798,
 109559,
 3108,
 95318,
 126121,
 113085,
 114495,
 115974,
 11328,
 111170,
 106185,
 38643,
 116966,
 51393,
 110235,
 122404,
 94690,
 95459,
 37302,
 106887,
 115243,
 14281,
 2826,
 103429,
 111809,
 104006,
 86341,
 23326,
 22416,
 117677,
 4159,
 110490,
 116852,
 33439,
 2027,
 66389,
 81617,
 88829,
 111773,
 103249,
 112154,
 88833,
 94893,
 11330,
 87541,
 118866,
 27107,
 98857,
 88666,
 38256,
 53961,
 1826,
 98182,
 110545,
 104140,
 53297,
 101603,
 95574,
 98163,
 34285,
 116429,
 2531,
 129124,
 98213,
 124783,
 119552,
 110548,
 63901,
 116508]

In [0]:
url = 'https://www.glowpick.com/beauty/ranking?id=15&level=2&brand_category_id='
get_product_numbers(url) #2

개수:  100


[104614,
 110707,
 98255,
 112261,
 112104,
 118912,
 128737,
 103199,
 81047,
 104736,
 116597,
 80672,
 84572,
 117455,
 97637,
 116438,
 125302,
 124797,
 125537,
 124841,
 102741,
 123519,
 80926,
 121315,
 123936,
 124922,
 94771,
 124792,
 111008,
 122425,
 1827,
 57493,
 93074,
 78704,
 128090,
 103431,
 105574,
 98930,
 125224,
 79818,
 14340,
 120209,
 110249,
 101721,
 80454,
 101350,
 105938,
 98049,
 125387,
 110809,
 120310,
 116576,
 122745,
 107716,
 127638,
 125038,
 77364,
 94470,
 40374,
 106621,
 131198,
 100504,
 92284,
 113065,
 123588,
 91900,
 105710,
 125882,
 129991,
 116238,
 124787,
 123057,
 96967,
 94630,
 86417,
 124949,
 93885,
 124988,
 123017,
 130479,
 90472,
 87686,
 26085,
 121510,
 108803,
 71929,
 84857,
 90207,
 93909,
 110492,
 116785,
 124221,
 96659,
 90761,
 94691,
 85605,
 110994,
 110984,
 99099,
 122270]

In [0]:
url = 'https://www.glowpick.com/beauty/ranking?id=19&level=2&brand_category_id='
get_product_numbers(url) #3

개수:  100


[1831,
 113165,
 157,
 53517,
 21274,
 2785,
 16899,
 128740,
 27571,
 99022,
 99221,
 81907,
 19043,
 55701,
 73251,
 110506,
 70173,
 46289,
 94867,
 26451,
 64869,
 1829,
 120915,
 44001,
 2986,
 110510,
 92171,
 82132,
 90935,
 82381,
 19944,
 61201,
 49013,
 21726,
 111305,
 141,
 8661,
 20253,
 101737,
 86960,
 1842,
 15522,
 18082,
 18086,
 127675,
 21275,
 82722,
 21495,
 114945,
 21493,
 84295,
 33616,
 81054,
 97907,
 15647,
 24677,
 24129,
 1839,
 25049,
 87288,
 34018,
 84299,
 5332,
 80914,
 40056,
 86896,
 90932,
 15,
 68933,
 42561,
 102564,
 58177,
 54937,
 47849,
 12240,
 12954,
 26716,
 27882,
 83795,
 21,
 83283,
 5153,
 11978,
 111202,
 56033,
 97191,
 50637,
 20689,
 43716,
 3625,
 35211,
 112527,
 27553,
 98184,
 128542,
 91062,
 20687,
 7404,
 73444,
 20252]

In [0]:
url = 'https://www.glowpick.com/beauty/ranking?id=22&level=2&brand_category_id='
get_product_numbers(url) #1

개수:  100


[267,
 2138,
 88952,
 81644,
 103360,
 42633,
 99987,
 104325,
 110386,
 63009,
 121976,
 71511,
 3403,
 474,
 124811,
 5905,
 85044,
 88953,
 2898,
 43392,
 124220,
 129309,
 111021,
 89142,
 90131,
 109301,
 124812,
 111178,
 93065,
 75130,
 18685,
 88954,
 106791,
 116786,
 91861,
 20191,
 97528,
 90129,
 204,
 2911,
 7234,
 2866,
 97243,
 92151,
 102587,
 132794,
 105795,
 92213,
 4448,
 34216,
 56013,
 124739,
 90912,
 117752,
 63881,
 92122,
 82498,
 116463,
 117342,
 92422,
 75131,
 89811,
 124955,
 93360,
 103715,
 95366,
 113006,
 100049,
 123310,
 99989,
 92912,
 99437,
 89847,
 271,
 106344,
 16868,
 92123,
 121226,
 88057,
 113674,
 90910,
 33722,
 129986,
 2913,
 89728,
 72635,
 95929,
 89182,
 116519,
 112418,
 72115,
 93355,
 80349,
 2794,
 98208,
 21230,
 129414,
 14001,
 69293,
 117649]

In [0]:
url = 'https://www.glowpick.com/beauty/ranking?id=20&level=2&brand_category_id='
get_product_numbers(url) #2

개수:  100


[48757,
 125380,
 109932,
 35754,
 478,
 35826,
 541,
 72955,
 117336,
 74211,
 110059,
 90665,
 106773,
 57121,
 71963,
 81753,
 109123,
 40449,
 103955,
 37515,
 40359,
 78143,
 106000,
 85811,
 112402,
 114598,
 51317,
 101595,
 110165,
 37962,
 91405,
 85087,
 74093,
 45937,
 5855,
 101669,
 94706,
 76835,
 90705,
 101517,
 92234,
 21237,
 14453,
 18593,
 128096,
 27242,
 88834,
 92119,
 25046,
 40296,
 83406,
 36660,
 45153,
 34091,
 37407,
 25662,
 534,
 3039,
 90400,
 90666,
 76608,
 93701,
 33520,
 13990,
 35520,
 51293,
 75126,
 38535,
 33842,
 6557,
 83068,
 3554,
 11972,
 98972,
 106535,
 91230,
 6559,
 76607,
 73313,
 79325,
 1889,
 106566,
 427,
 57185,
 7136,
 92280,
 74464,
 114203,
 90715,
 104405,
 113775,
 26394,
 116496,
 87281,
 77583,
 25661,
 40323,
 7962,
 14352,
 99731]

In [0]:
url = 'https://www.glowpick.com/beauty/ranking?id=21&level=2&brand_category_id='
get_product_numbers(url)

개수:  100


[433,
 48185,
 46273,
 79094,
 80406,
 75869,
 542,
 81218,
 75867,
 3167,
 39849,
 41559,
 60526,
 44485,
 79899,
 361,
 11968,
 112579,
 9421,
 75868,
 94962,
 429,
 8141,
 78913,
 251,
 107804,
 20009,
 60913,
 75874,
 27083,
 71853,
 552,
 45525,
 25050,
 35718,
 75870,
 12530,
 90282,
 108145,
 119889,
 58397,
 12162,
 20535,
 50589,
 66693,
 93639,
 47413,
 7077,
 34914,
 1880,
 90281,
 4453,
 88431,
 34615,
 7237,
 12516,
 72175,
 97947,
 89140,
 21537,
 100649,
 79085,
 88075,
 93253,
 41202,
 25660,
 112367,
 1964,
 88469,
 51413,
 60569,
 79095,
 96817,
 8604,
 441,
 91124,
 108501,
 66737,
 111011,
 38568,
 90874,
 6551,
 235,
 12513,
 91401,
 36993,
 11944,
 76075,
 50061,
 33730,
 3542,
 121609,
 83452,
 5575,
 102359,
 62209,
 86879,
 9100,
 124493,
 59381]

In [0]:
url = 'https://www.glowpick.com/beauty/ranking?id=23&level=2&brand_category_id='
get_product_numbers(url) #6

개수:  100


[118118,
 92409,
 123616,
 78557,
 118119,
 129699,
 24678,
 106908,
 33543,
 11099,
 540,
 27697,
 97647,
 120452,
 129713,
 121950,
 42831,
 106909,
 88033,
 50101,
 108843,
 80693,
 94483,
 111259,
 130341,
 34281,
 88253,
 73810,
 530,
 7335,
 130343,
 40089,
 39606,
 37251,
 114153,
 110058,
 75937,
 73809,
 230,
 107812,
 7442,
 4910,
 23602,
 110166,
 2546,
 50853,
 7349,
 80860,
 13707,
 52365,
 76289,
 2075,
 11969,
 5114,
 34224,
 98615,
 103100,
 8598,
 11970,
 74706,
 89046,
 76831,
 88161,
 21665,
 71949,
 19496,
 124010,
 18255,
 255,
 51537,
 247,
 122784,
 11615,
 123617,
 119706,
 40008,
 103502,
 6463,
 46981,
 90497,
 6343,
 98195,
 34354,
 80363,
 67513,
 121694,
 14284,
 80404,
 80012,
 97646,
 12050,
 76022,
 73202,
 88372,
 55413,
 498,
 2968,
 71269,
 82298,
 566]

In [0]:
url = 'https://www.glowpick.com/beauty/ranking?id=26&level=2&brand_category_id='
get_product_numbers(url) #8

개수:  100


[6340,
 110209,
 14329,
 99990,
 637,
 113988,
 1756,
 104967,
 664,
 103774,
 122587,
 10586,
 116598,
 573,
 735,
 72914,
 92159,
 89159,
 64773,
 2777,
 121312,
 14872,
 58809,
 84843,
 123851,
 60486,
 98968,
 73424,
 6231,
 64353,
 90906,
 11979,
 77215,
 36459,
 14875,
 102740,
 86668,
 715,
 97686,
 45545,
 39735,
 14874,
 89865,
 43401,
 71261,
 784,
 92190,
 94951,
 62205,
 89143,
 34638,
 62149,
 74734,
 105988,
 109354,
 603,
 38088,
 84553,
 2962,
 88908,
 88929,
 78290,
 129302,
 91939,
 1771,
 109762,
 12482,
 98232,
 18696,
 43458,
 93351,
 27490,
 36225,
 64769,
 90291,
 98774,
 111716,
 117340,
 128811,
 87959,
 14452,
 128638,
 1772,
 14208,
 110327,
 11333,
 104022,
 105660,
 97809,
 54241,
 1795,
 110613,
 5284,
 90907,
 88928,
 7347,
 89521,
 102852,
 98368,
 59859]

In [0]:
url = 'https://www.glowpick.com/beauty/ranking?id=27&level=2&brand_category_id='
get_product_numbers(url)

개수:  100


[124507,
 103772,
 15279,
 89222,
 94237,
 125188,
 127854,
 90947,
 72293,
 97503,
 123893,
 96883,
 90914,
 888,
 78787,
 78228,
 109436,
 81056,
 76807,
 83872,
 76220,
 14876,
 72298,
 725,
 101958,
 109373,
 106629,
 103452,
 74021,
 48081,
 118752,
 96678,
 80487,
 93801,
 78631,
 115552,
 78277,
 77424,
 78273,
 103106,
 102920,
 104351,
 77932,
 82629,
 79598,
 7125,
 112868,
 98251,
 79407,
 76511,
 108901,
 117620,
 88731,
 99304,
 75956,
 19842,
 92161,
 131756,
 90949,
 99130,
 43965,
 124583,
 85307,
 104365,
 88562,
 99978,
 124397,
 86434,
 92339,
 90180,
 126077,
 104368,
 93856,
 119459,
 131988,
 122052,
 115310,
 89938,
 98504,
 113123,
 80044,
 88378,
 81552,
 97685,
 117516,
 123434,
 647,
 129745,
 100039,
 102756,
 124140,
 126347,
 96974,
 104186,
 74655,
 668,
 42843,
 27582,
 73177,
 62297]

In [0]:
url = 'https://www.glowpick.com/beauty/ranking?id=49&level=2&brand_category_id='
get_product_numbers(url)

개수:  100


[18721,
 38244,
 49581,
 49861,
 111281,
 5259,
 20273,
 18723,
 3742,
 5258,
 111402,
 110557,
 22440,
 19008,
 81851,
 12940,
 10173,
 23364,
 1469,
 5244,
 13634,
 50605,
 98022,
 1258,
 82932,
 88901,
 120980,
 26159,
 27297,
 127445,
 10022,
 13146,
 5245,
 21502,
 44057,
 35556,
 4634,
 19570,
 10028,
 4627,
 10019,
 10025,
 50709,
 5904,
 42723,
 59083,
 85361,
 17157,
 85962,
 19941,
 27180,
 45741,
 20828,
 17156,
 21501,
 4200,
 23367,
 48429,
 77055,
 45941,
 42348,
 1480,
 64105,
 25220,
 84171,
 54549,
 20697,
 71989,
 26153,
 89116,
 24217,
 12945,
 10026,
 21500,
 16839,
 100573,
 57865,
 20662,
 77822,
 23378,
 5911,
 2613,
 90225,
 26770,
 84971,
 7912,
 19609,
 20462,
 49937,
 121242,
 59586,
 11806,
 76780,
 13684,
 95390,
 11223,
 8207,
 28759,
 4145,
 10031]

# 컨실러

In [0]:
url = 'https://www.glowpick.com/beauty/ranking?id=9&level=2&brand_category_id='
prod_list = get_product_numbers(url)
dt3 = multi_glo_crawler(prod_list)

개수:  100
제품명:  소프트 매트 컴플리트 컨실러
브랜드명:  나스 (NARS)
평균 점수:  4.41
리뷰 개수:  775
가져온 리뷰 개수:  0
제품명:  래디언트 크리미 컨실러
브랜드명:  나스 (NARS)
평균 점수:  4.27
리뷰 개수:  3161
가져온 리뷰 개수:  100
제품명:  디파이닝 커버 컨실러
브랜드명:  웨이크메이크 (WAKEMAKE)
평균 점수:  4.18
리뷰 개수:  2319
가져온 리뷰 개수:  100
제품명:  스튜디오 컨실러 앤 코렉트 팔레트
브랜드명:  맥 (MAC)
평균 점수:  4.25
리뷰 개수:  97
가져온 리뷰 개수:  97
제품명:  더블웨어 플로리스 컨실러
브랜드명:  에스티 로더 (ESTEE LAUDER)
평균 점수:  4.18
리뷰 개수:  293
가져온 리뷰 개수:  100
제품명:  하드커버 리퀴드 컨실러 [SPF30/PA++]
브랜드명:  홀리카홀리카 (HOLIKA HOLIKA)
평균 점수:  4.13
리뷰 개수:  177
가져온 리뷰 개수:  100
제품명:  킬 커버 에어리핏 컨실러
브랜드명:  클리오 (CLIO)
평균 점수:  4.01
리뷰 개수:  1271
가져온 리뷰 개수:  100
제품명:  테이핑 컨실러 롱웨어
브랜드명:  에스쁘아 (espoir)
평균 점수:  4.07
리뷰 개수:  399
가져온 리뷰 개수:  100
제품명:  풀 커버
브랜드명:  메이크업포에버 (MAKE UP FOR EVER)
평균 점수:  4.09
리뷰 개수:  316
가져온 리뷰 개수:  100
제품명:  더블 롱 웨어 커버 컨실러
브랜드명:  페리페라 (peripera)
평균 점수:  4.09
리뷰 개수:  68
가져온 리뷰 개수:  68
제품명:  더마 웨어 컨실러 [SPF30/PA++]
브랜드명:  더샘 (the SAEM)
평균 점수:  4.05
리뷰 개수:  65
가져온 리뷰 개수:  65
제품명:  마이 컨실러 [다크서클 커버]
브랜드명:  이니스프리 (innisfree)
평균 점수:  4.01

가져온 리뷰 개수:  18
제품명:  UHD 컨실러
브랜드명:  메이크업포에버 (MAKE UP FOR EVER)
평균 점수:  4.62
리뷰 개수:  13
가져온 리뷰 개수:  13
제품명:  릴렉싱 컨실러 스테이 선 세이프 [SPF30/PA++]
브랜드명:  헉슬리 (Huxley)
평균 점수:  3.69
리뷰 개수:  48
가져온 리뷰 개수:  48
제품명:  커버 퍼펙션 팁 컨실러 [SPF28/PA++]
브랜드명:  더샘 (the SAEM)
평균 점수:  3.54
리뷰 개수:  8691
가져온 리뷰 개수:  100
제품명:  미네랄라이징 크리미 컨실러 [SPF30/PA++]
브랜드명:  더샘 (the SAEM)
평균 점수:  3.56
리뷰 개수:  498
가져온 리뷰 개수:  100
제품명:  미네랄라이즈 컨실러
브랜드명:  맥 (MAC)
평균 점수:  3.77
리뷰 개수:  39
가져온 리뷰 개수:  39
제품명:  워터풀 컨실러
브랜드명:  3CE (쓰리씨이)
평균 점수:  3.79
리뷰 개수:  38
가져온 리뷰 개수:  38
제품명:  프로 핏 리퀴드 컨실러
브랜드명:  포니이펙트 (PONY EFFECT)
평균 점수:  3.88
리뷰 개수:  33
가져온 리뷰 개수:  33
제품명:  에이지 리와인드 컨실러
브랜드명:  메이블린 뉴욕 (MAYBELLINE NEWYORK)
평균 점수:  3.42
리뷰 개수:  1230
가져온 리뷰 개수:  20
제품명:  인텐시브 스킨 세럼 컨실러
브랜드명:  바비브라운 (BOBBI BROWN)
평균 점수:  3.66
리뷰 개수:  47
가져온 리뷰 개수:  47
제품명:  퍼펙팅 스틱 컨실러
브랜드명:  시세이도 (SHISEIDO)
평균 점수:  4.29
리뷰 개수:  14
가져온 리뷰 개수:  14
제품명:  쉐이프 테이프 컨투어 컨실러
브랜드명:  타르트 (Tarte)
평균 점수:  4.42
리뷰 개수:  12
가져온 리뷰 개수:  12
제품명:  롱웨어 컨실러 워터프루프
브랜드명:  아르데코 (ARTDECO)


In [0]:
dt3.to_csv('C://Users//secho//Desktop//glowpick_concealer.csv', index=False, encoding = 'utf-8-sig')

# 마스카라

In [0]:
prod_list = [118118,
 92409,
 123616,
 78557,
 118119,
 129699,
 24678,
 106908,
 33543,
 11099,
 540,
 27697,
 97647,
 120452,
 129713,
 121950,
 42831,
 106909,
 88033,
 50101,
 108843,
 80693,
 94483,
 111259,
 130341,
 34281,
 88253,
 73810,
 530,
 7335,
 130343,
 40089,
 39606,
 37251,
 114153,
 110058,
 75937,
 73809,
 230,
 107812,
 7442,
 4910,
 23602,
 110166,
 2546,
 50853,
 7349,
 80860,
 13707,
 52365,
 76289,
 2075,
 11969,
 5114,
 34224,
 98615,
 103100,
 8598,
 11970,
 74706,
 89046,
 76831,
 88161,
 21665,
 71949,
 19496,
 124010,
 18255,
 255,
 51537,
 247,
 122784,
 11615,
 123617,
 119706,
 40008,
 103502,
 6463,
 46981,
 90497,
 6343,
 98195,
 34354,
 80363,
 67513,
 121694,
 14284,
 80404,
 80012,
 97646,
 12050,
 76022,
 73202,
 88372,
 55413,
 498,
 2968,
 71269,
 82298,
 566]

In [0]:
dt = multi_glo_crawler(prod_list)

제품명:  킬래쉬 수퍼프루프 마스카라 [롱 컬링]
브랜드명:  클리오 (CLIO)
평균 점수:  4.4
리뷰 개수:  1760
가져온 리뷰 개수:  80
제품명:  엑스퍼트 아이밤 부스팅 픽서
브랜드명:  VDL (브이디엘)
평균 점수:  4.47
리뷰 개수:  148
가져온 리뷰 개수:  100
제품명:  래쉬 코렉팅 마스카라 [하이퍼 컬링]
브랜드명:  홀리카홀리카 (HOLIKA HOLIKA)
평균 점수:  4.48
리뷰 개수:  225
가져온 리뷰 개수:  100
제품명:  아이돌 리얼 래쉬 픽서
브랜드명:  아리따움 (ARITAUM)
평균 점수:  4.3
리뷰 개수:  2664
가져온 리뷰 개수:  100
제품명:  킬래쉬 수퍼프루프 마스카라 [볼륨 컬링]
브랜드명:  클리오 (CLIO)
평균 점수:  4.33
리뷰 개수:  366
가져온 리뷰 개수:  100
제품명:  스튜디오 더블 핏 마스카라 [볼륨 컬링]
브랜드명:  더샘 (the SAEM)
평균 점수:  4.35
리뷰 개수:  66
가져온 리뷰 개수:  66
제품명:  속눈썹 영양제 아이래쉬 세럼
브랜드명:  시드물 (SIDMOOL)
평균 점수:  4.18
리뷰 개수:  537
가져온 리뷰 개수:  100
제품명:  잉크 블랙 카라 AD [롱래쉬 컬링]
브랜드명:  페리페라 (peripera)
평균 점수:  4.14
리뷰 개수:  3366
가져온 리뷰 개수:  100
제품명:  오 마이 래쉬 쌩얼 카라
브랜드명:  에뛰드 (ETUDE)
평균 점수:  4.12
리뷰 개수:  9313
가져온 리뷰 개수:  100
제품명:  제트 컬링 마스카라
브랜드명:  라네즈 (LANEIGE)
평균 점수:  4.15
리뷰 개수:  558
가져온 리뷰 개수:  100
제품명:  히로인 메이크 롱앤컬 마스카라 EX
브랜드명:  키스미 (KISS ME)
평균 점수:  4.13
리뷰 개수:  12835
가져온 리뷰 개수:  100
제품명:  히로인 메이크 롱앤컬 마스카라 슈퍼 워터프루프
브랜드명:  키스미 (KISS M

In [0]:
dt.to_csv('C://Users//secho//Desktop//glowpick_7.csv', index=False, encoding = 'utf-8-sig')

# 립스틱

In [0]:
prod_list = [49097,
 111771,
 80915,
 97495,
 85963,
 71933,
 9783,
 21586,
 116516,
 117343,
 117648,
 128938,
 95456,
 90,
 96648,
 80276,
 110168,
 89691,
 93182,
 91,
 13971,
 110211,
 85036,
 117886,
 98856,
 124954,
 4924,
 62977,
 130647,
 78818,
 129123,
 124798,
 109559,
 3108,
 95318,
 126121,
 113085,
 114495,
 115974,
 11328,
 111170,
 106185,
 38643,
 116966,
 51393,
 110235,
 122404,
 94690,
 95459,
 37302,
 106887,
 115243,
 14281,
 2826,
 103429,
 111809,
 104006,
 86341,
 23326,
 22416,
 117677,
 4159,
 110490,
 116852,
 33439,
 2027,
 66389,
 81617,
 88829,
 111773,
 103249,
 112154,
 88833,
 94893,
 11330,
 87541,
 118866,
 27107,
 98857,
 88666,
 38256,
 53961,
 1826,
 98182,
 110545,
 104140,
 53297,
 101603,
 95574,
 98163,
 34285,
 116429,
 2531,
 129124,
 98213,
 124783,
 119552,
 110548,
 63901,
 116508]

In [0]:
dt = multi_glo_crawler(prod_list)

제품명:  럭스 립 칼라
브랜드명:  바비브라운 (BOBBI BROWN)
평균 점수:  4.38
리뷰 개수:  2491
가져온 리뷰 개수:  100
제품명:  글램락 미스티 로즈
브랜드명:  투쿨포스쿨 (too cool for school)
평균 점수:  4.54
리뷰 개수:  68
가져온 리뷰 개수:  68
제품명:  압솔뤼 루즈
브랜드명:  랑콤 (LANCOME)
평균 점수:  4.36
리뷰 개수:  1792
가져온 리뷰 개수:  100
제품명:  글램락 허쉬 브라운
브랜드명:  투쿨포스쿨 (too cool for school)
평균 점수:  4.41
리뷰 개수:  487
가져온 리뷰 개수:  100
제품명:  매드 매트 립
브랜드명:  클리오 (CLIO)
평균 점수:  4.33
리뷰 개수:  4113
가져온 리뷰 개수:  27
제품명:  너리싱 립 칼라
브랜드명:  바비브라운 (BOBBI BROWN)
평균 점수:  4.29
리뷰 개수:  1016
가져온 리뷰 개수:  100
제품명:  루즈 아르마니
브랜드명:  조르지오아르마니 (GIORGIO ARMANI)
평균 점수:  4.38
리뷰 개수:  284
가져온 리뷰 개수:  100
제품명:  르 루즈
브랜드명:  지방시 뷰티 (Givenchy Beauty)
평균 점수:  4.4
리뷰 개수:  166
가져온 리뷰 개수:  100
제품명:  데어 루즈 벨벳
브랜드명:  미샤 (MISSHA)
평균 점수:  4.36
리뷰 개수:  398
가져온 리뷰 개수:  100
제품명:  누드롭 듀이스틱
브랜드명:  홀리카홀리카 (HOLIKA HOLIKA)
평균 점수:  4.45
리뷰 개수:  65
가져온 리뷰 개수:  65
제품명:  스무스 매트 립스틱
브랜드명:  라카 (LAKA)
평균 점수:  4.34
리뷰 개수:  160
가져온 리뷰 개수:  100
제품명:  페이드 매트 립스틱
브랜드명:  레어카인드 (RAREKIND)
평균 점수:  4.28
리뷰 개수:  181
가져온 리뷰 개수:  100
제품명:  립컷 루즈 벨벳

In [0]:
dt.to_csv('C:/Users/secho/Desktop/glowpick_1.csv', index=False, encoding = 'utf-8-sig')